In [1]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    [[ -a myfile ]] && rm myfile
    [[ -a errors ]] && rm errors 
    [[ -a mydate ]] && rm mydate
    [[ -a mynewdir ]] && rm -rf mynewdir
    [[ -a anotherfile ]] && rm anotherfile
    [[ -a mybin ]] && rm -rf  mybin
    [[ -a myinfo ]] && rm myinfo''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()

generated="~/myfile ~/errors ~/mydate ~/mydir ~/mynewdir ~/out"


In [2]:
appdir=os.getenv('HOME')
appdir=appdir + "/lec3"
TermShellCmd("ls ")
output = runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";cp -r ../src/lec3 " + appdir)

# demke - moved starting bash session to here, so it can start with its cwd set to appdir
bash = BashSession(cwd=appdir)

In [3]:
bash.run("man 2 write", height='1in')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [4]:
bash.run("echo \"Hello class\"", height='1in')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [5]:
bash.run("echo \"Hello class\" > /tmp/reshello")

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [6]:
bash.run('''cat /tmp/reshello''')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [7]:
bash.run("ls /usr/bin/ | grep perl | wc -l")

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [8]:
bash.run('''ls -l /proc/$$/fd/{0..2}''')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [9]:
bash_pid=bash.getPid()
bash_stdout=os.path.realpath("/proc/" + bash_pid.__str__() + "/fd/1")
bash.run("file " + bash_stdout)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [10]:
bash.run("echo \"hello class\" > " + bash_stdout)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [11]:
bash.run(
    '''bash
bash
bash
ps -jH''')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [12]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#
# FOR SOME REASON doesn't work to specify language as c or c++ when generating book, for correct want

#display(Markdown('<font size="1.2rem">' + FileCodeBox(
#    file=appdir + "/testfork.c", 
#    lang="c", 
#    number=True,
#    title="<b>C: testfork.c",
#    h="100%", 
#    w="100%"
# ) + '</font>'))

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/testfork.c", 
    lang="", 
    number=True,
    title="<b>C: testfork.c</b>",
    h="100%", 
    w="100%"
) + '</font>'))
TermShellCmd("[[ -a testfork ]] && rm testfork;make testfork", cwd=appdir, prompt='', noposttext=True)
TermShellCmd("./testfork", cwd=appdir, noposttext=True)

<font size="1.2rem"><b>C: testfork.c</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: #include <sys/types.h>
1: #include <unistd.h>
2: #include <stdio.h>
3: #include <stdlib.h>
4: 
5: int
6: main(int argc, char* argv[]) 
7: {
8:   pid_t mypid, cpid, ppid;
9:   cpid = fork();
10:   ppid = getppid();
11:   mypid = getpid();
12: 
13:   if (cpid > 0) {
14:     /* Parent code */ 
15:     printf("hello, from parent with pid %d, pid of child is %d\n", mypid, cpid);
16:   } else if (cpid == 0) {
17:     /* Child code */
18:     printf("hello, I am child with pid %d, my parent is %d\n", mypid, ppid);
19:   } else {
20:     perror("fork failed\n");
21:     exit(-1);
22:   }
23:   return 0;
24: } 

```


</div>
</font>

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [13]:
bash.runNoOutput("[[ -a testfork ]] && rm testfork;make testfork")
bash.run("./testfork")

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [14]:
#This cell is removed in the html, but displays the code in the Jupyter notebook.
display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/doforke.c", 
    lang="", 
    number=True,
    title="<b> doforke.c: Illustrate use of fork and exec </b>",
    h="100%", 
    w="100%"
) + '</font>'))

<font size="1.2rem"><b> doforke.c: Illustrate use of fork and exec </b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: #include <sys/types.h>
1: #include <unistd.h>
2: #include <stdio.h>
3: #include <stdlib.h>
4: #include <sys/wait.h>
5: #include "doforke.h"
6: 
7: 
8: int
9: do_fork_exec(char *prog, char *const argv[])
10: {
11:   pid_t cpid;
12:   int status=0;
13:   cpid = fork();
14:   if (cpid < 0) {
15:     perror("fork failed\n");
16:     exit(-1);
17:   }
18: 
19:   if (cpid != 0) {
20:     // parent code, we need to wait for child
21:     waitpid(cpid,&status,0);
22:   } else {
23:     execve(prog, argv, 0);
24:     perror("should never get here\n");
25:   }
26:   return status;
27: } 

```


</div>
</font>

In [15]:
bash.run(
    '''echo $PATH
type cat''')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [16]:
bash.run("man 3 exec", height='1in')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [17]:
bash.run("pwd", height='1in')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [18]:
# sigh this doesn't work in book either
# This cell is removed in the html, but displays the code listing in the Jupyter notebook. 
file = appdir + "/pipe.c"
text_file = open(file, "r")
data = text_file.read()
data = numberLines(data)
text_file.close()                                                                                             
md_text = '''                                                                                                                        
``` ''' + "c" + '''                                                                                                                     
''' + data + '''                                                                                                                         
```                                                                                                                                      
'''
display (Markdown(md_text))
#display (Markdown('<font size=".5rem">' + md_text +'</font>'))

                                                                                                                        
``` c                                                                                                                     
0: #include <sys/types.h>
1: #include <sys/wait.h>
2: #include <stdio.h>
3: #include <stdlib.h>
4: #include <unistd.h>
5: #include <string.h>
6: 
7: int
8: main(int argc, char *argv[])
9: {
10:   int pipefd[2];
11:   pid_t cpid;
12:   char buf;
13:   
14:   if (argc < 2) {
15:     fprintf(stderr, "Usage: %s <string>\n", argv[0]);
16:     exit(EXIT_FAILURE);
17:   }
18:   
19:   if (pipe(pipefd) == -1) {
20:     perror("pipe");
21:     exit(EXIT_FAILURE);
22:   }
23:   
24:   cpid = fork();
25:   if (cpid == -1) {
26:     perror("fork");
27:     exit(EXIT_FAILURE);
28:   }
29:   
30:   if (cpid == 0) {    /* Child reads from pipe */
31:     // close(pipefd[1]);          /* Close unused write end */
32:     
33:     while (read(pipefd[0], &buf, 1) > 0)
34:       write(STDOUT_FILENO, &buf, 1);
35:     
36:     write(STDOUT_FILENO, "\n", 1);
37:     close(pipefd[0]);
38:     _exit(EXIT_SUCCESS);
39:     
40:   } else {            /* Parent writes argv[1] to pipe */
41:     close(pipefd[0]);          /* Close unused read end */
42:     write(pipefd[1], argv[1], strlen(argv[1]));
43:     close(pipefd[1]);          /* Reader will see EOF */
44:     wait(NULL);                /* Wait for child */
45:     exit(EXIT_SUCCESS);
46:   }
47: }
                                                                                                                         
```                                                                                                                                      


In [19]:
bash.run("ls /usr/bin/ | grep perl | wc -l")

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [20]:
# demke - I think it is potentially confusing to show a command here with two '&'s in it -- one for stderr redirection and one for background execution.
#bash.run("make >& RES &")
bash.run("make > RES &")

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…